<a href="https://colab.research.google.com/github/gopalchettri/DeepLearning/blob/master/2_a_Transfer_Learning_and_Fine_tuning_Pretrained_Model_using_Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Basic Transfer Learning using Tensorflow by downloading the MobileNetV2 Model  from Tensorflow Hub**

In [ ]:
"""
Transfer-learning pipeline using a frozen MobileNetV2 feature extractor
and a small custom classification head.

The script:
1. Imports libraries
2. Loads the TF-Hub backbone
3. Builds and compiles the model
4. Prepares augmented image generators
5. Trains, plots metrics, saves, reloads, and predicts
"""

# -----------------------------------------------------------
# 1. Library Imports
# -----------------------------------------------------------
import tensorflow as tf                      # Core deep-learning framework
import tensorflow_hub as hub                 # Provides the TF-Hub model zoo
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt              # For metric visualisation
import numpy as np                           # For numerical operations in prediction

print("[INFO] All libraries imported ✔")

# -----------------------------------------------------------
# 2. Load Pre-trained MobileNetV2 Feature Vector
# -----------------------------------------------------------
MODEL_URL   = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"
INPUT_SHAPE = (224, 224, 3)                  # Expected (H, W, C) by backbone

# hub.KerasLayer wraps a TF-Hub module so it behaves like a Keras layer.
mobilenet_feature_extractor = hub.KerasLayer(
    MODEL_URL,                               # Remote location of weights
    input_shape=INPUT_SHAPE,                 # Fix input tensor size
    trainable=False)                         # Freeze to keep backbone weights static

print("[INFO] MobileNetV2 backbone loaded & frozen ✔")

# -----------------------------------------------------------
# 3. Build the Classifier Head on top of the Frozen Backbone
# -----------------------------------------------------------
model = Sequential([
    mobilenet_feature_extractor,             # (0) Pre-trained convolutional base
    GlobalAveragePooling2D(),                # (1) Converts feature map → 1280-D vector
    Dense(256, activation='relu'),           # (2) Fully-connected layer
    Dense(128, activation='relu'),           # (3) Another FC layer
    Dense(10,  activation='softmax')         # (4) Softmax → probabilities for 10 classes
])

print("[INFO] Model architecture created ✔")

# -----------------------------------------------------------
# 4. Compile the Model
# -----------------------------------------------------------
model.compile(optimizer='adam',              # Adam optimiser (adaptive learning rate)
              loss='sparse_categorical_crossentropy', # Integer labels → sparse
              metrics=['accuracy'])          # Track classification accuracy

print("[INFO] Model compiled; here is the summary:")
model.summary()

# -----------------------------------------------------------
# 5. Data Preparation with Augmentation
# -----------------------------------------------------------
DATA_DIR   = "path/to/your/dataset"          # Root folder with sub-dirs = class names
BATCH_SIZE = 32                              # Number of images per mini-batch

# ImageDataGenerator performs on-the-fly data augmentation & normalisation
datagen = ImageDataGenerator(
    rescale=1./255,                          # Scale pixel values to [0, 1]
    rotation_range=20,                       # Random rotates ±20°
    width_shift_range=0.2,                   # Random horizontal shifts
    height_shift_range=0.2,                  # Random vertical shifts
    shear_range=0.2,                         # Shear transformations
    zoom_range=0.2,                          # Random zooms
    horizontal_flip=True,                    # Random horizontal flips
    validation_split=0.20)                   # 20 % data reserved for validation

# ----- Training subset generator -----
train_generator = datagen.flow_from_directory(
    DATA_DIR,
    target_size=INPUT_SHAPE[:2],             # Resize images to 224×224
    batch_size=BATCH_SIZE,
    class_mode='sparse',                     # Integer labels expected by loss fn
    subset='training')                       # Use training partition

# ----- Validation subset generator -----
val_generator = datagen.flow_from_directory(
    DATA_DIR,
    target_size=INPUT_SHAPE[:2],
    batch_size=BATCH_SIZE,
    class_mode='sparse',
    subset='validation')                     # Use validation partition

print(f"[INFO] Data generators ready – "
      f"{train_generator.samples} train images | "
      f"{val_generator.samples} val images ✔")

# -----------------------------------------------------------
# 6. Train the Model
# -----------------------------------------------------------
EPOCHS = 10
history = model.fit(
    train_generator,
    steps_per_epoch   = train_generator.samples // BATCH_SIZE,
    validation_data   = val_generator,
    validation_steps  = val_generator.samples // BATCH_SIZE,
    epochs=EPOCHS)

print("[INFO] Training finished ✔")

# -----------------------------------------------------------
# 7. Plot Accuracy & Loss Curves
# -----------------------------------------------------------
plt.figure(figsize=(12, 4))

# --- Accuracy subplot ---
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'],     label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.title('Accuracy per Epoch')
plt.xlabel('Epoch');  plt.ylabel('Accuracy');  plt.legend()

# --- Loss subplot ---
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'],     label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Loss per Epoch')
plt.xlabel('Epoch');  plt.ylabel('Loss');  plt.legend()

plt.tight_layout()
plt.show()

# -----------------------------------------------------------
# 8. Save the Trained Model
# -----------------------------------------------------------
MODEL_PATH = "mobilenet_transfer_learning_model"
model.save(MODEL_PATH)
print(f"[INFO] Model saved to: {MODEL_PATH} ✔")

# -----------------------------------------------------------
# 9. Reload the Model & Run a Demo Prediction
# -----------------------------------------------------------
loaded_model = tf.keras.models.load_model(
    MODEL_PATH,
    custom_objects={'KerasLayer': hub.KerasLayer})  # Needed to deserialize hub layer
print("[INFO] Saved model re-loaded ✔")

# ----- Prepare / load a single image -----
# Replace this random tensor with actual preprocessing of a real image
sample_image = np.random.rand(1, *INPUT_SHAPE).astype(np.float32)

# ----- Inference -----
probabilities = loaded_model.predict(sample_image)  # Output shape: (1, 10)
predicted_class_idx = np.argmax(probabilities, axis=1)[0]

print(f"[INFO] Predicted class index → {predicted_class_idx}")
